In [1]:
import pandas as pd
import numpy as np

# 1 DataFrameの確認

In [2]:
sample_df = pd.read_csv("csv_data/sample_df.csv")
sample_df.head()

,StudentID,Japanese,Math,Sex,Class
0,10,31.0,11,Female,B
1,11,72.0,69,Male,C
2,12,4.0,24,Male,B
3,13,71.0,15,Female,C
4,14,22.0,64,Male,A


In [3]:
sample_df.shape

(100, 5)

In [4]:
sample_df.dtypes

StudentID      int64
Japanese     float64
Math           int64
Sex           object
Class         object
dtype: object

# 2 統計量の確認

## 2.1 全体の統計量

In [5]:
sample_df.describe()

,StudentID,Japanese,Math
count,100.000000,85.000000,100.000000
mean,59.500000,49.611765,53.700000
std,29.011492,30.354294,28.084828
min,10.000000,1.000000,2.000000
25%,34.750000,24.000000,29.500000
50%,59.500000,50.000000,54.500000
75%,84.250000,77.000000,78.500000
max,109.000000,99.000000,99.000000


In [6]:
print(sample_df.count(numeric_only=True))
print("\n", sample_df.mean(numeric_only=True))
print("\n", sample_df.max(numeric_only=True))

StudentID    100
Japanese      85
Math         100
dtype: int64

 StudentID    59.500000
Japanese     49.611765
Math         53.700000
dtype: float64

 StudentID    109.0
Japanese      99.0
Math          99.0
dtype: float64


In [7]:
sample_df["Sex"].value_counts()

Male      60
Female    40
Name: Sex, dtype: int64

In [8]:
sample_df["Class"].value_counts()

C    30
A    27
B    25
Name: Class, dtype: int64

## 2.2 グループごとの統計量

In [9]:
sample_df.groupby("Sex").max(numeric_only=True)

,StudentID,Japanese,Math
Sex,,,
Female,108,99.0,99
Male,109,98.0,99


In [10]:
sample_df.groupby("Sex")["Japanese"].max(numeric_only=True)

Sex
Female    99.0
Male      98.0
Name: Japanese, dtype: float64

In [11]:
sample_df.groupby(["Sex", "Class"]).mean(numeric_only=True)

StudentID   Japanese       Math
Sex    Class                                 
Female A      55.230769  45.625000  60.923077
       B      51.272727  48.100000  58.636364
       C      66.727273  47.900000  48.090909
Male   A      56.285714  42.300000  63.642857
       B      57.071429  51.071429  52.571429
       C      63.368421  63.764706  49.473684

In [12]:
sample_df.groupby("Class")[["Japanese", "Math"]].agg([np.size, np.mean, np.sum])

Japanese                    Math                 
          size       mean     sum size       mean   sum
Class                                                  
A           27  43.777778   788.0   27  62.333333  1683
B           25  49.833333  1196.0   25  55.240000  1381
C           30  57.888889  1563.0   30  48.966667  1469

In [13]:
sample_func = lambda x: str(round(np.mean(x), 2)) + "点"
sample_df.groupby("Class")[["Japanese", "Math"]].agg([sample_func, np.mean, np.sum])

Japanese                          Math                 
      <lambda_0>       mean     sum <lambda_0>       mean   sum
Class                                                          
A         43.78点  43.777778   788.0     62.33点  62.333333  1683
B         49.83点  49.833333  1196.0     55.24点  55.240000  1381
C         57.89点  57.888889  1563.0     48.97点  48.966667  1469

In [14]:
sample_func = lambda x: str(round(np.mean(x), 2)) + "点"
sample_df.groupby("Class").agg({"Japanese": [sample_func, np.sum], "Math": [np.mean, np.std]})

Japanese               Math           
      <lambda_0>     sum       mean        std
Class                                         
A         43.78点   788.0  62.333333  30.157280
B         49.83点  1196.0  55.240000  28.994655
C         57.89点  1563.0  48.966667  27.513612

# 3 欠損値処理

## 3.1 欠損数確認

In [15]:
sample_df.isnull()

,StudentID,Japanese,Math,Sex,Class
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
95,False,False,False,False,True
96,False,False,False,False,False
97,False,False,False,False,False
98,False,False,False,False,False


In [16]:
sample_df.isnull().sum()

StudentID     0
Japanese     15
Math          0
Sex           0
Class        18
dtype: int64

In [17]:
sample_df.isnull().sum()/len(sample_df)*100

StudentID     0.0
Japanese     15.0
Math          0.0
Sex           0.0
Class        18.0
dtype: float64

In [18]:
percent_func = lambda x: str(round(x, 2)) + "%"
(sample_df.isnull().sum()/len(sample_df)*100).apply(percent_func)

StudentID     0.0%
Japanese     15.0%
Math          0.0%
Sex           0.0%
Class        18.0%
dtype: object

In [19]:
sample_df.isnull().sum().sort_values(ascending=False).head(3)

Class        18
Japanese     15
StudentID     0
dtype: int64

In [20]:
sample_df.isnull().sum()[sample_df.isnull().sum() > 0].sort_values(ascending=False)

Class       18
Japanese    15
dtype: int64

## 3.2 欠損データ確認

In [21]:
sample_df[sample_df.isnull().any(axis=1)].head()

,StudentID,Japanese,Math,Sex,Class
6,16,58.0,47,Male,NaN
7,17,NaN,4,Female,B
15,25,NaN,15,Female,A
17,27,16.0,16,Female,NaN
23,33,NaN,30,Male,A


In [22]:
sample_df[(sample_df["Sex"]=="Male") & (sample_df.isnull().any(axis=1))].head()

,StudentID,Japanese,Math,Sex,Class
6,16,58.0,47,Male,NaN
23,33,NaN,30,Male,A
29,39,NaN,77,Male,A
31,41,NaN,60,Male,A
33,43,84.0,77,Male,NaN


## 3.3 欠損値補完

In [23]:
sample_df.fillna(50).head(7)

,StudentID,Japanese,Math,Sex,Class
0,10,31.0,11,Female,B
1,11,72.0,69,Male,C
2,12,4.0,24,Male,B
3,13,71.0,15,Female,C
4,14,22.0,64,Male,A
5,15,50.0,22,Male,B
6,16,58.0,47,Male,50


In [24]:
complement_df = sample_df.fillna({"Japanese": sample_df["Japanese"].mean(), "Class": sample_df["Class"].mode()[0]})
complement_df.head()

,StudentID,Japanese,Math,Sex,Class
0,10,31.0,11,Female,B
1,11,72.0,69,Male,C
2,12,4.0,24,Male,B
3,13,71.0,15,Female,C
4,14,22.0,64,Male,A


In [25]:
compliment_f = lambda x: x.fillna(x.mean())
sample_df.groupby("Sex")["Japanese"].apply(compliment_f)[[5,13]]

5     50.0
13    53.0
Name: Japanese, dtype: float64

In [26]:
sample_df[(sample_df["Sex"]=="Male") & (sample_df["Japanese"].isnull())].head()

,StudentID,Japanese,Math,Sex,Class
23,33,NaN,30,Male,A
29,39,NaN,77,Male,A
31,41,NaN,60,Male,A
74,84,NaN,24,Male,C
87,97,NaN,12,Male,C


In [27]:
sample_df[(sample_df["Sex"]=="Female") & (sample_df["Japanese"].isnull())].head()

,StudentID,Japanese,Math,Sex,Class
7,17,NaN,4,Female,B
15,25,NaN,15,Female,A
30,40,NaN,13,Female,A
42,52,NaN,99,Female,A
51,61,NaN,28,Female,NaN


# 4 変数の変形

In [28]:
complement_df.head()

,StudentID,Japanese,Math,Sex,Class
0,10,31.0,11,Female,B
1,11,72.0,69,Male,C
2,12,4.0,24,Male,B
3,13,71.0,15,Female,C
4,14,22.0,64,Male,A


## 4.1 新しい変数の作成

In [29]:
complement_df["J+M"] = complement_df["Japanese"] + complement_df["Math"]
complement_df.head()

,StudentID,Japanese,Math,Sex,Class,J+M
0,10,31.0,11,Female,B,42.0
1,11,72.0,69,Male,C,141.0
2,12,4.0,24,Male,B,28.0
3,13,71.0,15,Female,C,86.0
4,14,22.0,64,Male,A,86.0


## 4.3 ワンホットエンコーディング

In [30]:
complement_df["Sex"] = complement_df["Sex"].replace({"Male": 1, "Female": 0})
complement_df.head()

,StudentID,Japanese,Math,Sex,Class,J+M
0,10,31.0,11,0,B,42.0
1,11,72.0,69,1,C,141.0
2,12,4.0,24,1,B,28.0
3,13,71.0,15,0,C,86.0
4,14,22.0,64,1,A,86.0


In [31]:
pd.get_dummies(complement_df, columns=["Class"]).head()

,StudentID,Japanese,Math,Sex,J+M,Class_A,Class_B,Class_C
0,10,31.0,11,0,42.0,0,1,0
1,11,72.0,69,1,141.0,0,0,1
2,12,4.0,24,1,28.0,0,1,0
3,13,71.0,15,0,86.0,0,0,1
4,14,22.0,64,1,86.0,1,0,0


In [32]:
onehot_df = pd.get_dummies(complement_df, columns=["Class"]).iloc[:, :-1]
onehot_df.head()

,StudentID,Japanese,Math,Sex,J+M,Class_A,Class_B
0,10,31.0,11,0,42.0,0,1
1,11,72.0,69,1,141.0,0,0
2,12,4.0,24,1,28.0,0,1
3,13,71.0,15,0,86.0,0,0
4,14,22.0,64,1,86.0,1,0


# 5 データの保存

In [33]:
onehot_df.to_csv("csv_data/preprocess_df.csv", index_label=False)

In [34]:
pd.read_csv("csv_data/preprocess_df.csv")

,StudentID,Japanese,Math,Sex,J+M,Class_A,Class_B
0,10,31.0,11,0,42.0,0,1
1,11,72.0,69,1,141.0,0,0
2,12,4.0,24,1,28.0,0,1
3,13,71.0,15,0,86.0,0,0
4,14,22.0,64,1,86.0,1,0
...,...,...,...,...,...,...,...
95,105,25.0,82,1,107.0,0,0
96,106,83.0,3,1,86.0,0,0
97,107,98.0,86,1,184.0,0,1
98,108,3.0,94,0,97.0,0,1
